In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import torch

from torch import nn
from torch import optim
from torch.utils.data import DataLoader

In [3]:
from cbow.dataset import CBOWDataset, DataContainer
from cbow.model import CBOWClassifier, ModelContainer
from cbow.consts import consts
consts

Namespace(batch_size=128, checkpointer_name='cbow_classifier', checkpointer_prefix='cbow', device='cuda:3', early_stopping_criteria=5, embedding_size=50, learning_rate=0.0001, model_dir=PosixPath('../data/books/work_dir/models'), num_epochs=100, path=PosixPath('../data/books'), proc_dataset_csv=PosixPath('../data/books/frankenstein_with_splits.csv'), save_every=2, save_total=5, vectorizer_file=PosixPath('../data/books/work_dir/vectorizer.json'), work_dir=PosixPath('../data/books/work_dir'))

In [4]:
df = pd.read_csv(consts.proc_dataset_csv)
dc = DataContainer(df, consts.vectorizer_file, consts.batch_size, is_load=True)

In [5]:
classifier = CBOWClassifier(dc.vocabulary_size, consts.embedding_size)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=consts.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', factor=0.5, patience=1)

mc = ModelContainer(classifier, optimizer, loss_func, scheduler)